In [1]:
import pandas as pd
import numpy as np
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
# gensim
import gensim
from gensim.corpora import Dictionary
from gensim import models
from gensim.parsing.preprocessing import STOPWORDS

# plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

# nltk
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *


In [2]:
# topic modelling
news = pd.read_csv("Cleaned Data/headlines.csv", encoding = "ISO-8859-1")
news.head()

,date,source,headline
0,2008-10-14,business_daily,Trade financing drought hits Kenyan companies
1,2008-10-14,business_daily,House asks Treasury to table fresh budget pla
2,2008-10-14,business_daily,Kenya gets harsh human rights verdic
3,2008-10-14,business_daily,Ripple effects of global crisis set to hit Eas...
4,2008-10-14,business_daily,Manufacturers cut product sizes to beat recessio


In [3]:
news["headline1"] = news["headline"].str.lower()
news["headline1"] = news["headline1"].str.replace("[^a-z\\s]", " ")
news["headline1"] = news["headline1"].str.replace("\\s+", " ")
news["headline1"].head()

0        trade financing drought hits kenyan companies
1        house asks treasury to table fresh budget pla
2                 kenya gets harsh human rights verdic
3    ripple effects of global crisis set to hit eas...
4     manufacturers cut product sizes to beat recessio
Name: headline1, dtype: object

In [4]:
# tokenization
news["headline1"] = news["headline1"].apply(word_tokenize)
news["headline1"].head()

0    [trade, financing, drought, hits, kenyan, comp...
1    [house, asks, treasury, to, table, fresh, budg...
2          [kenya, gets, harsh, human, rights, verdic]
3    [ripple, effects, of, global, crisis, set, to,...
4    [manufacturers, cut, product, sizes, to, beat,...
Name: headline1, dtype: object

In [5]:
lemmatizer = WordNetLemmatizer()
news["headline1"] = news["headline1"].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
news["headline1"].head()

0    [trade, financing, drought, hit, kenyan, company]
1    [house, asks, treasury, to, table, fresh, budg...
2            [kenya, get, harsh, human, right, verdic]
3    [ripple, effect, of, global, crisis, set, to, ...
4    [manufacturer, cut, product, size, to, beat, r...
Name: headline1, dtype: object

In [6]:
english_stopwords = stopwords.words('english')
english_stopwords.extend(["news", "live blog", "live",  "in pictures", "download", "blog",
   "lifestyle", "editorial", "editorials", "pictures", "opinion", "read",
  "newsplex", "politics", "new", "news live", "photos",  "video", 
  "frontrow", "fact check", "financial time", "watch live", "columnists", "interview", "habari", "kenya", "kenyan", "star"])
news["headline1"] = news["headline1"].apply(lambda x: [y for y in x if y not in english_stopwords and  len(y) > 2])
news["headline1"].head()                        

0            [trade, financing, drought, hit, company]
1    [house, asks, treasury, table, fresh, budget, ...
2                   [get, harsh, human, right, verdic]
3    [ripple, effect, global, crisis, set, hit, eas...
4    [manufacturer, cut, product, size, beat, reces...
Name: headline1, dtype: object

In [7]:
# create bag of words
dictionary = Dictionary(news["headline1"]) # dictionary of 51127 words
dictionary.filter_extremes(no_below = 5, no_above = 0.5) # dictionary of 18063 words


In [8]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count == 10:
        break


0 company
1 drought
2 financing
3 hit
4 trade
5 asks
6 budget
7 fresh
8 house
9 pla


In [9]:
bow = [dictionary.doc2bow(headline) for headline in news["headline1"]]
dictionary[bow[0][0][0]]

'company'

In [10]:
lda_model = gensim.models.LdaMulticore(bow, num_topics=3, id2word=dictionary, passes=2, workers=2, per_word_topics = True, random_state = 0)

In [11]:
pprint(lda_model.print_topics())

[(0,
  '0.008*"market" + 0.008*"firm" + 0.007*"bank" + 0.007*"company" + '
  '0.006*"africa" + 0.006*"deal" + 0.006*"county" + 0.006*"state" + '
  '0.006*"plan" + 0.005*"farmer"'),
 (1,
  '0.010*"man" + 0.010*"woman" + 0.009*"police" + 0.007*"year" + 0.006*"two" + '
  '0.005*"death" + 0.005*"life" + 0.004*"killed" + 0.004*"attack" + '
  '0.004*"nairobi"'),
 (2,
  '0.012*"uhuru" + 0.009*"get" + 0.008*"ruto" + 0.008*"raila" + 0.008*"court" '
  '+ 0.007*"say" + 0.006*"governor" + 0.006*"leader" + 0.005*"president" + '
  '0.005*"world"')]


In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow, dictionary,  sort_topics = False)
vis

C:\Users\Chance\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.233834 -0.170223       1        1  33.204533
1     -0.281962 -0.095766       2        1  32.111397
2      0.048128  0.265988       3        1  34.684071, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
641   Default  7115.000000      uhuru  7115.000000  30.0000  30.0000
431   Default  5531.000000        man  5531.000000  29.0000  29.0000
139   Default  4565.000000     police  4565.000000  28.0000  28.0000
81    Default  4645.000000     market  4645.000000  27.0000  27.0000
12    Default  4914.000000        get  4914.000000  26.0000  26.0000
203   Default  4672.000000      raila  4672.000000  25.0000  25.0000
547   Default  4702.000000       ruto  4702.000000  24.0000  24.0000
613   Default  5743.000000      woman  5743.000000  23.0000  23.0000
226   Default  4120.000000       firm  4120.000000  22.0000  22.0000
842   Default  4038.000000       bank  4038.000000  21.0000  21.0000
307   Default  4584.000000      court  4584.000000  20.0000  20.0000
0     Default  3714.000000    company  3714.000000  19.0000  19.0000
219   Default  3376.000000       deal  3376.000000  18.0000  18.0000
17    Default  3411.000000     africa  3411.000000  17.0000  17.0000
504   Default  3126.000000        two  3126.000000  16.0000  16.0000
2459  Default  3692.000000   governor  3692.000000  15.0000  15.0000
143   Default  3451.000000     leader  3451.000000  14.0000  14.0000
300   Default  3043.000000       plan  3043.000000  13.0000  13.0000
111   Default  2856.000000     farmer  2856.000000  12.0000  12.0000
732   Default  2746.000000      death  2746.000000  11.0000  11.0000
897   Default  2816.000000        tax  2816.000000  10.0000  10.0000
1871  Default  3146.000000  president  3146.000000   9.0000   9.0000
698   Default  2689.000000       life  2689.000000   8.0000   8.0000
972   Default  5218.000000       year  5218.000000   7.0000   7.0000
293   Default  2253.000000     killed  2253.000000   6.0000   6.0000
138   Default  2789.000000       team  2789.000000   5.0000   5.0000
725   Default  2203.000000     attack  2203.000000   4.0000   4.0000
2648  Default  2145.000000   arrested  2145.000000   3.0000   3.0000
3577  Default  2295.000000    jubilee  2295.000000   2.0000   2.0000
49    Default  2236.000000   business  2236.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
1746   Topic3   839.123352       anti   841.538025   1.0560  -6.5247
202    Topic3   741.089172    kalonzo   742.343872   1.0572  -6.6490
143    Topic3  3380.374512     leader  3451.226562   1.0381  -5.1313
307    Topic3  4442.209961      court  4584.145508   1.0274  -4.8582
15     Topic3  1349.024536      right  1361.668579   1.0496  -6.0499
2459   Topic3  3570.615234   governor  3692.009033   1.0255  -5.0766
1871   Topic3  3034.852051  president  3146.477539   1.0228  -5.2392
1029   Topic3  1444.018066      final  1465.200317   1.0443  -5.9819
138    Topic3  2694.731689       team  2789.885254   1.0242  -5.3580
470    Topic3  1408.987671        use  1432.811401   1.0421  -6.0064
888    Topic3  1641.687256       work  1684.237061   1.0333  -5.8536
2260   Topic3  1194.732300       fake  1209.951172   1.0462  -6.1714
1371   Topic3  1460.393555       free  1497.685181   1.0337  -5.9706
835    Topic3  1644.590210      money  1705.306519   1.0226  -5.8518
944    Topic3  2361.507568       tell  2586.575195   0.9679  -5.4900
1235   Topic3  1978.676880     census  2125.538330   0.9873  -5.6669
770    Topic3  1508.915771      title  1557.678223   1.0271  -5.9379
890    Topic3  1833.859619        law  2099.139893   0.9238  -5.7429
1203   Topic3  1830.856445      order  2107.192871   0.9183  -5.7445
167    Topic3  3794.764648        say  6307.520508   0.5508  -5.0157
48     Topic3  2696.061279    